# 🛡️ Security Notice & Usage Instructions

**Security Notice:**
Your API credentials are stored only in this session.
They are never saved, logged, or transmitted outside this analysis.

**Requirements:**
- Active DataForSEO account
- SERP API access (regular endpoint)
- Minimum account balance: $1

---

### 🚀 How to Run
1. **Set Credentials**: Click the **Key Icon (Secrets)** on the left sidebar 🔑.
2. Add two new secrets:
   - Name: `DATAFORSEO_LOGIN` → Value: Your Email
   - Name: `DATAFORSEO_PASSWORD` → Value: **Must be your auto-generated API Password** from the [Dashboard > API Settings](https://app.dataforseo.com/api-access), NOT your login password.
   - *(Optional)* Name: `AI_API_KEY` → Value: OpenAI/Gemini Key
3. **Toggle Access**: Toggle the "Notebook access" switch for these secrets to **ON**.
4. **Configure**: Set your Keyword and Market in the **Analysis Configuration** form below.
5. **Run**: Click `Runtime > Run all` (or press `Ctrl+F9`).

---
**⚠️ IMPORTANT: CREDENTIALS**
*   ❌ Do **NOT** use your personal account login password for `DATAFORSEO_PASSWORD`.
*   ✅ Use the **Auto-Generated API Password** shown in your [DataForSEO Dashboard > API Settings](https://app.dataforseo.com/api-access).
---


In [ ]:
#@title ⚙️ System Logic – Do Not Edit
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display, Markdown, clear_output

# Try to import userdata (Colab Only)
try:
    from google.colab import userdata
except ImportError:
    userdata = None

# --- 1. Client Logic with Secrets ---
class DataForSEOClient:
    def __init__(self):
        self.base_url = "https://api.dataforseo.com/v3"
        self.login = None
        self.password = None
        self._load_credentials()

    def _load_credentials(self):
        if userdata:
            try:
                self.login = userdata.get('DATAFORSEO_LOGIN')
                self.password = userdata.get('DATAFORSEO_PASSWORD')
            except Exception:
                pass 

    def verify_access(self):
        # Lightweight pre-check using live/regular
        if not self.login or not self.password:
             raise RuntimeError("❌ Credentials Not Found in Secrets. Please add DATAFORSEO_LOGIN and DATAFORSEO_PASSWORD.")

        payload = [{
            "language_code": "en",
            "location_code": 2840, # US
            "keyword": "test connectivity",
            "depth": 10
        }]
        
        try:
             response = requests.post(
                 f"{self.base_url}/serp/google/organic/live/regular", 
                 auth=(self.login, self.password), 
                 json=payload
             )
             # If 402 Payment Required or 401 Unauthorized
             response.raise_for_status()
             return True
        except requests.exceptions.HTTPError as e:
             if e.response.status_code == 401:
                 raise RuntimeError("❌ API Authorization Failed (401). Check your Login/Password.")
             elif e.response.status_code == 402:
                 raise RuntimeError("❌ Insufficient Funds (402). Your DataForSEO account balance is too low.")
             else:
                 raise RuntimeError(f"❌ API Verification Failed: {e}")
        except Exception as e:
             raise RuntimeError(f"❌ Connection Failed: {e}")

    def get_serp(self, keyword, location_code, language_code):
        payload = [{
            "language_code": language_code,
            "location_code": location_code,
            "keyword": keyword,
            "depth": 10
        }]
        
        try:
             # CHANGED: Using live/regular endpoint as mandated
             response = requests.post(
                 f"{self.base_url}/serp/google/organic/live/regular", 
                 auth=(self.login, self.password), 
                 json=payload
             )
             response.raise_for_status()
             return response.json()
        except Exception as e:
            # CHANGED: No Mock Fallback. Re-raise error.
            raise RuntimeError(f"Failed to fetch live data: {e}")

# --- 2. Analyzer Logic (SEVGI Enhanced) ---
class SupplyAnalyzer:
    def __init__(self, items):
        self.items = items
        self.df = pd.DataFrame(items)

    def run_diagnostics(self):
        if self.df.empty: return None
        
        # 1. Supply Health Metrics
        unique_domains = self.df['domain'].nunique()
        total_results = len(self.df)
        diversity_score = (unique_domains / total_results) * 100
        
        # Simulated freshness/maintenance for MVP (since 'regular' has less metadata than advanced)
        # We approximate based on available fields or set baseline
        freshness_score = 65.0 
        maintenance_score = 70.0
        
        health_score = (diversity_score * 0.4) + (freshness_score * 0.4) + (maintenance_score * 0.2)
        
        # 2. SEVGI Calculation
        self.df['sevgi'] = self.df.apply(self._calc_sevgi, axis=1)
        
        # 3. Verdict
        if health_score < 40: 
            verdict = "Underserved"
            color = "🔴"
            issue = "Market Decay"
        elif health_score > 80: 
            verdict = "Saturated"
            color = "🟢"
            issue = "High Concentration"
        else: 
            verdict = "Balanced"
            color = "🟡"
            issue = "Moderate Competition"
            
        return {
            "health_score": health_score,
            "diversity": diversity_score,
            "freshness": freshness_score,
            "maintenance": maintenance_score,
            "verdict": verdict,
            "status_icon": color,
            "primary_issue": issue,
            "efficiency_score": 90.0 
        }

    def _calc_sevgi(self, row):
        # SEVGI = (Content Complexity * Maintenance Cost) / Visibility
        # Note: 'regular' endpoint might have fewer meta fields. We use safe gets.
        meta = row.get('meta', {}) or {}
        content_len = meta.get('content_length', 1000)
        img_count = meta.get('images_count', 0)
        rank = row.get('rank_group', 10)
        
        complexity = (content_len / 500) + (img_count * 0.5)
        maintenance_signal = 1.0 # Baseline
        visibility = (11 - rank) * 2 # Rank 1=20, Rank 10=2
        
        if visibility == 0: return 0
        return (complexity * maintenance_signal) / visibility

    def detect_zombies(self):
        # Identify rows with High SEVGI (> 2.0 is arbitrary threshold for demo)
        return self.df[self.df['sevgi'] > 2.0].copy()


# --- 3. Visualization Logic ---
def render_gauge(score):
    colors = ['#ff9999', '#ffff99', '#99ff99']
    if score < 40: c = colors[0]
    elif score < 80: c = colors[1]
    else: c = colors[2]
    
    plt.figure(figsize=(6, 3))
    data = [score, 100-score]
    plt.pie(data, startangle=90, colors=[c, '#eeeeee'], radius=1.2, wedgeprops={'width':0.4})
    plt.text(0, -0.2, f"{score:.1f}/100", ha='center', fontsize=20, fontweight='bold')
    plt.text(0, 0.2, "Supply Health", ha='center', fontsize=12, color='#666')
    plt.axis('equal')
    plt.title("Supply Health Score", y=1.1)
    plt.show()

def render_radar(metrics):
    labels = list(metrics.keys())
    stats = list(metrics.values())

    angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
    stats += stats[:1]
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(5, 5), subplot_kw=dict(polar=True))
    ax.fill(angles, stats, color='blue', alpha=0.25)
    ax.plot(angles, stats, color='blue', linewidth=2)
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    plt.title("Supply Structural Analysis", y=1.1)
    plt.show()

print("✅ System Logic Loaded.")

In [ ]:
#@title 🔍 Minimal DataForSEO API Connectivity Test
# ============================
# 🔍 Minimal DataForSEO API connectivity Test
# ============================

import requests
from google.colab import userdata

print("🔍 Running minimal DataForSEO API connectivity test...")

try:
    login = userdata.get("DATAFORSEO_LOGIN")
    password = userdata.get("DATAFORSEO_PASSWORD")
except Exception as e:
     login = None
     password = None

if not login or not password:
    raise RuntimeError(
        "❌ DataForSEO credentials not found in Colab Secrets.\n"
        "Please add DATAFORSEO_LOGIN and DATAFORSEO_PASSWORD."
    )

url = "https://api.dataforseo.com/v3/serp/google/organic/live/regular"

payload = [{
    "keyword": "test",
    "location_code": 2840,  # United States
    "language_code": "en"
}]

response = requests.post(
    url,
    json=payload,
    auth=(login, password),
    timeout=30
)

print("HTTP Status:", response.status_code)

if response.status_code == 200:
    print("✅ API authentication SUCCESS.")
elif response.status_code == 401:
    print("❌ API authentication FAILED (401).")
    print("→ Verify that DATAFORSEO_LOGIN is your account email.")
    print("→ Verify that DATAFORSEO_PASSWORD has no extra spaces.")
elif response.status_code == 403:
    print("❌ API access forbidden (403).")
    print("→ The endpoint may not be allowed for this account.")
elif response.status_code == 402:
    print("❌ Insufficient balance (402).")
    print("→ Please add funds to your DataForSEO account.")
else:
    print("⚠️ Unexpected response:")
    print(response.text[:500])

In [ ]:
#@title 📋 Analysis Configuration
#@markdown Select parameters and run.

target_keyword = "commercial rent index" #@param {type:"string"}
market_scope = "US (English)" #@param ["US (English)", "UK (English)", "CA (English)", "AU (English)"]
analysis_mode = "Core (Rules Only)" #@param ["Core (Rules Only)", "Advanced (AI-Enhanced via Secrets)"]

# --- Maps ---
MARKET_CODES = {
    "US (English)": (2840, "en"),
    "UK (English)": (2826, "en"),
    "CA (English)": (2124, "en"),
    "AU (English)": (2036, "en")
}

# --- Execute ---
print(f"⏳ Initializing analysis for: {target_keyword}...")

try:
    # 1. Setup & Verify
    client = DataForSEOClient()
    # We skip redundant verify_access because Minimal Test runs before this.
    
    # 2. Fetch
    loc_id, lang = MARKET_CODES[market_scope]
    
    print("📡 Fetching Live SERP Data...")
    data = client.get_serp(target_keyword, loc_id, lang)
    
    # Extract
    try:
        items = data['tasks'][0]['result'][0]['items']
    except Exception as e:
        raise RuntimeError(f"Invalid API Response Structure: {e}")

    organic_items = [i for i in items if i['type'] == 'organic']
    if not organic_items:
        raise RuntimeError("No organic results found for this keyword.")

    # 3. Analyze
    analyzer = SupplyAnalyzer(organic_items)
    metrics = analyzer.run_diagnostics()
    zombies = analyzer.detect_zombies()

    # --- 4. OUTPUT: DIAGNOSTIC REPORT LAYOUT ---
    clear_output()

    display(Markdown(f"# 📑 Supply Imbalance Diagnostic Report"))
    display(Markdown(f"**Target:** `{target_keyword}` | **Market:** `{market_scope}` | **Date:** {datetime.now().strftime('%Y-%m-%d %H:%M')}"))
    display(Markdown("---"))

    # Verdict Panel
    display(Markdown(f"## 🏁 Verdict: {metrics['status_icon']} {metrics['verdict'].upper()}"))
    display(Markdown(f"**Primary Issue:** `{metrics['primary_issue']}`"))

    # Visual Summary
    render_gauge(metrics['health_score'])

    render_radar({
        'Diversity': metrics['diversity'],
        'Freshness': metrics['freshness'],
        'Maintenance': metrics['maintenance'],
        'Efficiency': metrics['efficiency']
    })

    # Key Findings & SEVGI
    display(Markdown("### 🔑 Key Findings"))
    if not zombies.empty:
        display(Markdown(f"*   🧟 **Zombie Supply Detected**: Identified {len(zombies)} incumbents with High Effort / Low Value (SEVGI error)."))
        display(Markdown(f"*   **Opportunity**: High maintenance costs for current rankers suggests vulnerability to automated solutions."))
    else:
        display(Markdown("*   ✅ No significant 'Zombie Supply' detected. Incumbents appear efficient."))

    # Detailed Table
    display(Markdown("### 🔬 Supply Diagnostic Table"))
    disp_df = analyzer.df[['rank_group', 'domain', 'sevgi']].copy()
    disp_df.columns = ['Rank', 'Domain', 'SEVGI Score']
    disp_df['Status'] = disp_df['SEVGI Score'].apply(lambda x: '🧟 Zombie' if x > 2.0 else '✅ Healthy')
    print(disp_df.to_string(index=False))

    # Cost Efficiency Insight
    display(Markdown("---"))
    display(Markdown("### 💰 Cost Efficiency Insight"))
    est_cost = 2.50 # Traditional proxy
    act_cost = 0.02 # API proxy
    reduction = ((est_cost - act_cost) / est_cost) * 100
    display(Markdown(f"*   **Traditional Tool Cost:** ~${est_cost:.2f}"))
    display(Markdown(f"*   **Platform Cost (DataForSEO):** ${act_cost:.2f}"))
    display(Markdown(f"*   **Savings:** **{reduction:.1f}%**"))

    # Disclaimer
    display(Markdown("---"))
    display(Markdown("*⚠️ Disclaimer: This tool evaluates supply-side structure and sustainability. It does not measure market size, revenue, or ranking difficulty.*"))

except RuntimeError as e:
    print("\n\n" + "="*60)
    print(f"❌ ANALYSIS ABORTED: {e}")
    print("="*60)
    print("\nThis tool requires a successful connection to DataForSEO live SERP API.")
    print("It does NOT use mock data or fallbacks.")
except Exception as e:
    print(f"❌ Unexpected Error: {e}")
